In [1]:
import json
import pprint

In [5]:
from collections import namedtuple
import pandas as pd

def convert_to_mb(bytes):
    return bytes/1048576

def convert_to_gb(bytes):
    return bytes/1073741824

files = [
    # 'logs/amz_category_bestsellers.log',
    # 'logs/amz_category_hot_new_releases.log',
    # 'logs/amz_category_movers_and_shakers.log',
    # 'logs/amz_category_most_wished_for.log',
    # 'logs/amz_category_urls_bestsellers.log',
    # 'logs/amz_category_urls_hot_new_releases.log',
    # 'logs/amz_category_urls_movers_and_shakers.log',
    # 'logs/amz_category_urls_most_wished_for.log',
    'logs/amz_product.log', 
    'logs/amzProductsLC_home-improvement.log',
    'logs/amzProductsLC_jewelry.log', 
    'logs/amzProductsLC_office.log',
    'logs/amz_lc_luggage.log',
    'logs/amzProductsLC_kitchen.log'
]
total_mbs = 0
total_gb = 0
Log = namedtuple('Log', ['file', 'run_count', 'item_count', 'data_transferred_mb', 'data_transferred_gb'])
logs = []
for file in files:
    print(file)
    run_count = 0
    response_bytes = 0
    request_bytes = 0
    item_count = 0
    with open(file, 'r') as f:
        lines = f.readlines()
        for line in lines:
            if 'downloader/response_bytes' in line:
                run_count += 1
                bytes = int(line.split(': ')[1].split(',')[0])
                response_bytes += bytes
            elif 'downloader/request_bytes' in line:
                bytes = int(line.split(': ')[1].split(',')[0])
                request_bytes += bytes
            elif 'item_scraped_count' in line:
                item_count += int(line.split(': ')[1].split(',')[0])
    
    log = Log(file.split('/')[-1], run_count, item_count, convert_to_mb(response_bytes)+convert_to_mb(request_bytes), convert_to_gb(response_bytes)+convert_to_gb(request_bytes))
    print(log)
    logs.append(log)
    total_mbs += log.data_transferred_mb
    total_gb += log.data_transferred_gb
    print("\n")

print("\n")
print(f'Total MBs: {total_mbs}')
print(f'Total GBs: {total_gb}')
logs_df = pd.DataFrame(logs)
# logs_df.to_csv('logs.csv', index=False)

logs_df

logs/amz_product.log
Log(file='amz_product.log', run_count=1, item_count=7642, data_transferred_mb=2148.698532104492, data_transferred_gb=2.098338410258293)


logs/amzProductsLC_home-improvement.log
Log(file='amzProductsLC_home-improvement.log', run_count=1, item_count=421, data_transferred_mb=4.580657958984375, data_transferred_gb=0.004473298788070679)


logs/amzProductsLC_jewelry.log
Log(file='amzProductsLC_jewelry.log', run_count=2, item_count=20335, data_transferred_mb=88.70634841918945, data_transferred_gb=0.0866272933781147)


logs/amzProductsLC_office.log
Log(file='amzProductsLC_office.log', run_count=0, item_count=0, data_transferred_mb=0.0, data_transferred_gb=0.0)


logs/amz_lc_luggage.log
Log(file='amz_lc_luggage.log', run_count=2, item_count=11373, data_transferred_mb=54.634796142578125, data_transferred_gb=0.05335429310798645)


logs/amzProductsLC_kitchen.log
Log(file='amzProductsLC_kitchen.log', run_count=1, item_count=52182, data_transferred_mb=403.69543743133545, data_t

,file,run_count,item_count,data_transferred_mb,data_transferred_gb
0,amz_product.log,1,7642,2148.698532,2.098338
1,amzProductsLC_home-improvement.log,1,421,4.580658,0.004473
2,amzProductsLC_jewelry.log,2,20335,88.706348,0.086627
3,amzProductsLC_office.log,0,0,0.000000,0.000000
4,amz_lc_luggage.log,2,11373,54.634796,0.053354
5,amzProductsLC_kitchen.log,1,52182,403.695437,0.394234


In [3]:
with open(".json/lc_pages.json") as f:
    data = json.load(f)

In [6]:
print(len(data))
print(data[0])

516
{'category': 'apparel', 'subcategory': '1968073031', 'avg_rating': '3.90', 'avg_sales_qty': '7000', 'total_pages': 209}


In [ ]:
# import asyncio
from helpers.utils import extract_paths
from helpers.postgres_handler_async import AsyncPostgresDBHandler 
from settings import POSTGRES_HOST, POSTGRES_DATABASE, POSTGRES_USERNAME, POSTGRES_PASSWORD, POSTGRES_PORT

In [ ]:
async def main(batch_size=10):
    items = list()
    db = AsyncPostgresDBHandler(POSTGRES_HOST, POSTGRES_DATABASE, POSTGRES_USERNAME, POSTGRES_PASSWORD, POSTGRES_PORT)
    await db.connect()
    keys = list()
    for d in data:
        items.append(d)
        if len(items) >= batch_size:
            await db.bulk_upsert('staging.stg__dummy', items, conflict_columns=['category', 'sub_category'])
            items = list()
    await db.close()
    

In [24]:
# pprint.pprint(data)
db.close()

In [15]:
import pandas as pd
import json

In [66]:
paths = []
with open('.json/categories_mapping3.json') as f:
    data = json.load(f)
output = extract_paths(data)
df = pd.DataFrame(output)
# filter out rows where lvl8 is None
df[df['lvl8'].notna()]


,category,lvl1,lvl2,lvl3,lvl4,lvl5,lvl6,lvl7,lvl8,data_node
13607,office,3591019031,3591063031,15759997031,3591014031,3591546031,3591552031,3591084031,3591270031,3591270031
13608,office,3591019031,3591063031,15759997031,3591014031,3591546031,3591552031,3591084031,3591272031,3591272031
13609,office,3591019031,3591063031,15759997031,3591014031,3591546031,3591552031,3591084031,3591273031,3591273031


In [29]:
paths[-10:]

[['watches', '5518819031', '5518821031'],
 ['watches', '5518819031', '5518820031'],
 ['watches', '5518819031'],
 ['watches', '2563504031', '15723938031'],
 ['watches', '2563504031', '15723937031'],
 ['watches', '2563504031'],
 ['watches', '2563505031', '15723942031'],
 ['watches', '2563505031', '15723939031'],
 ['watches', '2563505031', '15723941031'],
 ['watches', '2563505031']]

In [25]:
data_dict = extract_paths(data)

# if data_dict[0].get("category") == 'bestsellers':
#     data_dict.pop(0)

ValueError: max() iterable argument is empty